My household listens to a lot of Taylor Swift music.
Inspired by this and her Eras tour, I decided to take a look at some of her song data available on Spotify.
In this example, I'm visualizing the data with plotly.
These plots are more interactive, but will not render correctly in Github, so I would recommend viewing the notebook using nbviewer.

**Follow the link below to view the interactive plots!**

[nbviewer link](https://www.google.com)

In [6]:
import pandas as pd
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import plotly.express as px


In [7]:
df = pd.read_csv("data/Taylor Swift.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 57 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   added_at                            254 non-null    object 
 1   is_local                            254 non-null    bool   
 2   primary_color                       0 non-null      float64
 3   added_by.external_urls.spotify      254 non-null    object 
 4   added_by.href                       254 non-null    object 
 5   added_by.id                         254 non-null    object 
 6   added_by.type                       254 non-null    object 
 7   added_by.uri                        254 non-null    object 
 8   track.album.album_type              254 non-null    object 
 9   track.album.artists                 254 non-null    object 
 10  track.album.available_markets       254 non-null    object 
 11  track.album.external_urls.spotify   254 non-n

In [5]:
df['track.album.release_date'] = pd.to_datetime(df['track.album.release_date'])

In [11]:
figure = px.box(df, x="track.album.name", y="track.popularity", points='all', hover_name='track.name')
figure